# Interactive type checking with datashape

In [1]:
from datashape import *
from inspect import *
from functools import *
from toolz.curried import *
from toolz.curried.operator import *

In [2]:
def to_datashape(annotation):
    if isinstance(annotation, str):
        return dshape(annotation)
    if isinstance(annotation, dict):
        return Record(
            (_1, to_datashape(_2)) for _1, _2 in annotations.items())
    return from_numpy(tuple(), annotation)

In [3]:
def annotation_to_shapes(callable):
    shapes = pipe(
        callable, getfullargspec, attrgetter('annotations'),
        valmap(to_datashape)
    )
    return shapes, shapes.pop('return')

In [4]:
def check_shapes(values, shapes):
    type_vars = {}
    for key, shape in shapes.items():
        value = values.pop(key)
        discovered = discover(value)
        new_parameters = []
        for dim, parameter in enumerate(shape.parameters):
            if isinstance(parameter, TypeVar):                
                if parameter not in type_vars:
                    if dim == len(shape.parameters)-1:
                        new = discover(value).measure
                    else:
                        if hasattr(value, 'shape'):
                            new = Fixed(value.shape[dim])
                        else:
                            new = Fixed(len(value))
                    type_vars[parameter] = new
                new_parameters.append(type_vars[parameter])
            else:
                new_parameters.append(parameter)
                
        shape = DataShape(*new_parameters)
                            
        if not validate(shape, value):
            raise InteractiveTypeError(f"{key} expects {shape}, but recieved {discover(value)}")
            
    if values:
        Err(f"No types for {list(values)}.")

In [5]:
class InteractiveTypeError(BaseException):...

In [6]:
def typecheck(callable):
    @wraps(callable)
    def caller(*args, **kwargs):
        argspec = getfullargspec(callable)
        shapes, returns = annotation_to_shapes(callable)
        values = merge(
            dict(zip(argspec.args, argspec.defaults or [])),
            dict(zip(argspec.args, args)),
            kwargs)
        check_shapes(values, shapes)
        output = callable(*args, **kwargs)
        return output
    return caller

## Demonstrate on a simple dot function.

In [7]:
@typecheck
def dot(x: "N*float64", y: "N*float64") -> "float64":
    return sum(_1*_2 for _1, _2 in zip(x, y))

In [8]:
dot([10.], [20.])

200.0

## Demonstrate on matrix multiplication.

In [9]:
@typecheck
def matmul(x: "N*M*float64", y: "M*T*float64") -> "N*T*float64":
    return x@y

In [10]:
matmul(np.random.randn(10,4), np.random.randn(4, 2))

array([[ 1.43853722,  0.81275112],
       [ 2.41409867,  0.71916123],
       [ 0.78274515, -0.29016317],
       [-2.63409325, -1.76115546],
       [-1.25299493, -0.61953802],
       [ 2.25528122,  0.31669383],
       [-0.33032512,  0.04367129],
       [ 0.25128923, -0.72226858],
       [-0.53314789, -0.14783717],
       [ 1.88993265,  1.33088029]])

### Execute a function with invalid generic types.

In [11]:
matmul(np.random.randn(10,4), np.random.randn(3, 2))

InteractiveTypeError: y expects 4 * 2 * float64, but recieved 3 * 2 * float64